In [63]:
debug=False
name="programD"
codigo_fuente=open("../programs/asm/"+name+".s").read()

In [64]:
# ToDo

pseudo_instr = [
	"beqz", "bnez", "li", 
	"mv", "j", "jr", 
	"la", "neg", "nop", 
	"not", "ret", "seqz", 
	"snez", "bgt", "ble"
]


In [65]:
# 1. Dividir el código en líneas
lineas = codigo_fuente.split('\n')

# Definimos los caracteres que consideramos válidos
caracteres_validos = "abcdefghijklmnopqrstuvwxyz0123456789-(): "

lineas_limpias = []
for linea in lineas:
    # 2. Eliminar comentarios
    linea_sin_comentarios = linea.strip().split('#')[0].strip()

    # 3. Ignorar líneas que ahora están vacías
    if not linea_sin_comentarios:
        continue

    # 4. Reemplazar comas por espacios
    linea_sin_comas = linea_sin_comentarios.replace(',', ' ')

    # 5. Reemplazar dobles espacios por espacios simples de forma repetida
    linea_espacios_ok = linea_sin_comas
    while '  ' in linea_espacios_ok:
        linea_espacios_ok = linea_espacios_ok.replace('  ', ' ')

    # --- NUEVAS LÍNEAS ---
    # 6. Convertir todo a minúsculas
    linea_minusculas = linea_espacios_ok.lower()

    # 7. Eliminar caracteres no válidos
    linea_final = "".join(filter(lambda char: char in caracteres_validos, linea_minusculas))
    # --------------------

    lineas_limpias.append(linea_final.strip()) # strip() al final por si quedan espacios sobrantes

# Imprimir el resultado para verificar
if debug:
    for i, linea in enumerate(lineas_limpias):
        print(f"Línea {i}: '{linea}'")

In [66]:
# --- PASO 1: Detectar etiquetas con espacios y crear un mapa de corrección ---
mapeo_etiquetas = {}
for linea in lineas_limpias:
    if ':' in linea:
        # Extraemos solo la parte de la etiqueta, antes de los dos puntos
        etiqueta_original = linea.split(':')[0].strip()
        if ' ' in etiqueta_original:
            etiqueta_normalizada = etiqueta_original.replace(' ', '')
            print(f"⚠️ WARNING: La etiqueta '{etiqueta_original}' contiene espacios. Se normalizará a '{etiqueta_normalizada}'.")
            mapeo_etiquetas[etiqueta_original] = etiqueta_normalizada

# --- PASO 2: Aplicar la normalización a todo el código ---
lineas_corregidas = []
for linea in lineas_limpias:
    linea_actual = linea
    # Aplicamos todas las correcciones encontradas a cada línea
    for original, normalizada in mapeo_etiquetas.items():
        linea_actual = linea_actual.replace(original, normalizada)
    lineas_corregidas.append(linea_actual)

# --- PASO 3: Separar las etiquetas de sus instrucciones ---
lineas_finales = []
etiquetas=set()

for linea in lineas_corregidas:
    if ':' in linea:
        partes = linea.split(':', 1) # Dividir solo por el primer ':'
        etiqueta = partes[0].strip() 
        etiquetas.add(etiqueta)
        etiqueta=etiqueta + ':'
        instruccion = partes[1].strip()
        lineas_finales.append(etiqueta) # Añade la línea con la etiqueta
        if instruccion: # Si hay una instrucción, añádela en una nueva línea
            lineas_finales.append(instruccion)
    else:
        lineas_finales.append(linea) # Si no hay ':', la línea se queda como está

# Imprimir el resultado para verificar
if debug:
    print("\n--- Código Procesado ---")
    for i, linea in enumerate(lineas_finales):
        print(f"Línea {i}: '{linea}'")

In [67]:
# --- FASE 1: Cargar datos de instrucciones y registros ---
import io
# Usamos io.StringIO para leer los strings como si fueran archivos
instr_file = open("instruction_data.dat")
reg_file = open("reg_map.dat")

# Cargar instrucciones válidas (el primer campo de cada línea)
instrucciones_validas = set()
datos_instrucciones = []
for linea in instr_file:
    linea = linea.strip()
    if linea and not linea.startswith('#'):
        partes=linea.split()
        datos_instrucciones.append(partes)
        instrucciones_validas.add(partes[0])


# Cargar mapa de registros y nombres ABI válidos
mapa_registros = {}
registros_validos_abi = set()
for linea in reg_file:
    linea = linea.strip()
    if linea and not linea.startswith('#'):
        partes = linea.split()
        abi_name, arch_name = partes[0], partes[1]
        mapa_registros[abi_name] = arch_name
        registros_validos_abi.add(abi_name)


In [68]:
def isAddress(palabra, valid_registers, valid_labels):
    """
    Verifica si una palabra tiene el formato de dirección 'offset(registro)' o 'etiqueta(registro)'.
    """
    if '(' not in palabra or ')' not in palabra or palabra.find('(') > palabra.find(')'):
        return False
    try:
        partes = palabra.split('(')
        offset_o_etiqueta = partes[0]
        registro = partes[1].replace(')', '')
        if registro not in valid_registers:
            return False
        if offset_o_etiqueta.isnumeric() or (offset_o_etiqueta.startswith('-') and offset_o_etiqueta[1:].isnumeric()):
            return True
        if offset_o_etiqueta in valid_labels:
            return True
    except (IndexError, ValueError):
        return False
    return False

In [69]:
# --- FASE 2: Procesamiento de las líneas de código ---


def es_entero(cadena):
    """
    Devuelve True si la cadena puede convertirse a un entero, False en caso contrario.
    """
    try:
        int(cadena)
        return True
    except ValueError:
        return False
    
# 1º) Eliminar líneas con palabras no válidas
instrucciones_validas.add('nop')
palabras_permitidas = instrucciones_validas.union(registros_validos_abi).union(etiquetas)
lineas_paso1 = []
for linea in lineas_finales:
    if ':' in linea: # Las etiquetas pasan el filtro directamente
        lineas_paso1.append(linea)
        continue
    
    palabras = linea.split()
    linea_es_valida = True
    for palabra in palabras:
        # Una palabra es válida si es permitida O si es un número
        palabraNoReconocida=palabra not in palabras_permitidas
        if palabraNoReconocida and not es_entero(palabra) and not isAddress(palabra, registros_validos_abi, etiquetas):

            print(f"⚠️ WARNING: La palabra '{palabra}' no es válida. Se descarta la línea: '{linea}'")
            linea_es_valida = False
            break # No es necesario seguir revisando palabras en esta línea
    
    if linea_es_valida:
        lineas_paso1.append(linea)

# 2º) Eliminar líneas cuya primera palabra no sea una instrucción
lineas_paso2 = []
for linea in lineas_paso1:
    if ':' in linea: # Las etiquetas pasan el filtro
        lineas_paso2.append(linea)
        continue

    primera_palabra = linea.split()[0]
    if primera_palabra in instrucciones_validas:
        lineas_paso2.append(linea)
    # Las líneas que no cumplen la condición simplemente no se añaden

# 3º) Sustituir registros ABI por su nombre de arquitectura
lineas_paso3 = []
for linea in lineas_paso2:
    if ':' in linea: # Las etiquetas no se modifican
        lineas_paso3.append(linea)
        continue

    palabras = linea.split()

    palabras_sustituidas = []
    for p in palabras:
        if isAddress(p, registros_validos_abi, etiquetas):
            # Caso especial: es una dirección como '0(a0)'
            offset, reg_con_parentesis = p.split('(')
            registro_abi = reg_con_parentesis[:-1] # Quita el ')'
            registro_arch = mapa_registros.get(registro_abi, registro_abi)
            palabras_sustituidas.append(f"{offset}({registro_arch})")
        else:
            # Caso normal
            palabras_sustituidas.append(mapa_registros.get(p, p))    
    lineas_paso3.append(" ".join(palabras_sustituidas))

In [70]:
# --- Primera Pasada del Ensamblador ---

# Diccionario para guardar la tabla de símbolos (etiquetas y sus direcciones)
tabla_de_etiquetas = {}
# Lista para guardar únicamente las líneas que son instrucciones
codigo_instrucciones = []

# Nuestro contador de programa (PC), empieza en la dirección 0.
direccion_actual = 0
# Asumimos que cada instrucción ocupa 4 bytes.
INSTRUCTION_SIZE = 4

for linea in lineas_paso3:
    if linea.endswith(':'):
        # Es una etiqueta. La registramos con la dirección de la SIGUIENTE instrucción.
        nombre_etiqueta = linea[:-1] # Quitamos los dos puntos ':'
        tabla_de_etiquetas[nombre_etiqueta] = direccion_actual
    else:
        # Es una instrucción. La guardamos y avanzamos el contador de programa.
        codigo_instrucciones.append(linea)
        direccion_actual += INSTRUCTION_SIZE

# --- RESULTADO FINAL ---
if debug:
    print("--- Tabla de Etiquetas (Símbolos) ---")
    print(tabla_de_etiquetas)

    print("\n--- Código solo con Instrucciones ---")
    for i, instruccion in enumerate(codigo_instrucciones):
        print(f"Línea {i}: '{instruccion}'")

# --- Segunda Pasada del Ensamblador ---

# Lista para guardar el código final con las etiquetas reemplazadas por offsets
codigo_final = []

for i, instruccion in enumerate(codigo_instrucciones):
    # Calculamos la dirección de la instrucción actual
    pc = i * INSTRUCTION_SIZE
    
    partes = instruccion.split()
    # El operando a reemplazar suele ser el último (para saltos y branches)
    operando_a_revisar = partes[-1]
    
    if operando_a_revisar in tabla_de_etiquetas:
        # El operando es una etiqueta, hay que calcular el offset
        direccion_destino = tabla_de_etiquetas[operando_a_revisar]
        offset = direccion_destino - pc
        
        # Reemplazamos la etiqueta por el offset calculado
        partes[-1] = str(offset)
        linea_modificada = " ".join(partes)
        codigo_final.append(linea_modificada)
    else:
        # La instrucción no usa una etiqueta, se queda como está
        codigo_final.append(instruccion)

# --- RESULTADO FINAL ---
if debug:
    print("--- Código Final con Offsets Calculados ---")
    for i, linea in enumerate(codigo_final):
        # La dirección se calcula como i * 4 para mostrarla junto a la instrucción
        print(f"0x{i * INSTRUCTION_SIZE:08x}: '{linea}'")

In [71]:
# --- Funciones de Ensamblado (Tercera Pasada) ---

def ensamblarR(partes, instruccion_data):
    """Ensambla una instrucción de tipo R: opcode|rd|funct3|rs1|rs2|funct7"""
    rd  = int(partes[1][1:])   # asume formato xN
    rs1 = int(partes[2][1:])
    rs2 = int(partes[3][1:])
    opcode = int(instruccion_data[2],base=2)
    funct3 = int(instruccion_data[3],base=2)
    funct7 = int(instruccion_data[4],base=2)
    instr = (funct7 << 25) | (rs2 << 20) | (rs1 << 15) | (funct3 << 12) | (rd << 7) | opcode
    return instr

def ensamblarI(partes, instruccion_data):
    """Ensambla una instrucción de tipo I: opcode|rd|funct3|rs1|imm[11:0]"""
    rd  = int(partes[1][1:])
    try:
        rs1 = int(partes[2][1:])
        imm = int(partes[3], 0) & 0xFFF
    except:
        offset_str, reg_str = partes[2].split("(")
        imm = int(offset_str, 0) & 0xFFF
        rs1 = int(reg_str.strip(")")[1:])  # quitar ")" y la "x"
    

    opcode = int(instruccion_data[2],base=2)
    funct3 = int(instruccion_data[3],base=2)
    instr = (imm << 20) | (rs1 << 15) | (funct3 << 12) | (rd << 7) | opcode
    return instr

def ensamblarS(partes, instruccion_data):
    """S-type: opcode|imm[11:5]|rs2|rs1|funct3|imm[4:0]"""
    rs2 = int(partes[1][1:])

    offset_str, reg_str = partes[2].split("(")
    imm = int(offset_str, 0) & 0xFFF
    rs1 = int(reg_str.strip(")")[1:])  # quitar ")" y la "x"
    
    opcode = int(instruccion_data[2],base=2)
    funct3 = int(instruccion_data[3],base=2)
    instr = ((imm >> 5) << 25) | (rs2 << 20) | (rs1 << 15) | (funct3 << 12) | ((imm & 0x1F) << 7) | opcode
    return instr

def ensamblarB(partes, instruccion_data):
    """B-type: opcode|imm[12]|imm[10:5]|rs2|rs1|funct3|imm[4:1]|imm[11]"""
    rs1 = int(partes[1][1:])
    rs2 = int(partes[2][1:])
    imm = int(partes[3], 0) & 0x1FFF
    opcode = int(instruccion_data[2],base=2)
    funct3 = int(instruccion_data[3],base=2)
    instr = ((imm >> 12 & 1) << 31) | ((imm >> 5 & 0x3F) << 25) | (rs2 << 20) | (rs1 << 15) \
            | (funct3 << 12) | ((imm >> 1 & 0xF) << 8) | ((imm >> 11 & 1) << 7) | opcode
    return instr

def ensamblarSB(partes, instruccion_data):
    """Alias de ensamblarB (algunos manuales llaman SB a las branch)."""
    return ensamblarB(partes, instruccion_data)

def ensamblarU(partes, instruccion_data):
    """U-type: opcode|rd|imm[31:12]"""
    rd  = int(partes[1][1:])
    imm = int(partes[2], 0) & 0xFFFFF000
    opcode = int(instruccion_data[2],base=2)
    instr = (imm) | (rd << 7) | opcode
    return instr

def ensamblarJ(partes, instruccion_data):
    """J-type (JAL): opcode|rd|imm[20]|imm[10:1]|imm[11]|imm[19:12]"""
    rd  = int(partes[1][1:])
    imm = int(partes[2], 0) & 0x1FFFFF
    opcode = int(instruccion_data[2],base=2)
    instr = ((imm >> 20 & 1) << 31) | ((imm >> 1 & 0x3FF) << 21) \
            | ((imm >> 11 & 1) << 20) | ((imm >> 12 & 0xFF) << 12) \
            | (rd << 7) | opcode
    return instr

def ensamblarUJ(partes, instruccion_data):
    """Alias de ensamblarJ (algunos manuales dicen UJ para JAL)."""
    return ensamblarJ(partes, instruccion_data)



In [72]:
def calcularTipo(mnemonico, datos_instrucciones):
    """Busca una instrucción en nuestros datos y devuelve su formato (R, I, J, etc.)."""
    for instruccion_data in datos_instrucciones:
        if instruccion_data[0] == mnemonico:
            return instruccion_data[1] # El formato es el segundo campo
    return None # No se encontró la instrucción


def ensamblarInstruccion(instruccion_str, datos_instrucciones):
    """
    Toma una línea de código de texto y la convierte en su código máquina (un entero).
    """
    partes = instruccion_str.split()
    mnemonico = partes[0]
    
    # Caso especial para nop, como solicitaste
    if mnemonico == 'nop':
        return 0x00000013

    # 1. Averiguamos el formato de la instrucción (R, I, J...)
    tipo_instruccion = calcularTipo(mnemonico, datos_instrucciones)
    
    # Buscamos los datos completos de la instrucción (opcode, funct3, etc.)
    instruccion_data = next((data for data in datos_instrucciones if data[0] == mnemonico), None)

    # print(instruccion_data)

    if not tipo_instruccion or not instruccion_data:
        print(f"ERROR: Instrucción desconocida '{mnemonico}'")
        return 0 # Devolvemos 0 o un error

    # 2. Llamamos a la función de ensamblado correspondiente a su tipo
    if tipo_instruccion == 'R':
        return ensamblarR(partes, instruccion_data)
    elif tipo_instruccion == 'I':
        return ensamblarI(partes, instruccion_data)
    elif tipo_instruccion == 'S':
        return ensamblarS(partes, instruccion_data)
    elif tipo_instruccion == 'B':
        return ensamblarB(partes, instruccion_data)
    elif tipo_instruccion == 'SB':
        return ensamblarSB(partes, instruccion_data)
    elif tipo_instruccion == 'J':
        return ensamblarJ(partes, instruccion_data)
    elif tipo_instruccion == 'U':
        return ensamblarU(partes, instruccion_data)
    elif tipo_instruccion == 'UJ':
        return ensamblarUJ(partes, instruccion_data)
    else:
        print(f"ERROR: Tipo de instrucción '{tipo_instruccion}' no soportado.")
        return 0

def ensamblarCodigo(codigo_texto, datos_instrucciones):
    """
    Función principal que recorre todas las líneas de código y las ensambla.
    """
    codigo_maquina = []
    for instruccion_str in codigo_texto:
        palabra_maquina = ensamblarInstruccion(instruccion_str, datos_instrucciones)
        codigo_maquina.append(palabra_maquina)
    return codigo_maquina

# --- EJECUCIÓN DEL ENSAMBLADO ---
# (Esta línea la ejecutarías para probar que la estructura funciona)
codigo_maquina = ensamblarCodigo(codigo_final, datos_instrucciones)
# print(codigo_maquina)

In [73]:
program = b''.join(c.to_bytes(4, byteorder="little") for c in codigo_maquina)

salida=open("../programs/bin/"+name+".bin",'wb')
salida.write(program)
salida.close

print("PROGRAM_D = bytes([")
for machine_code, instruction_text in zip(codigo_maquina, codigo_final):
    b = machine_code.to_bytes(4, byteorder="little")
    byte_string = ", ".join(f"0x{byte:02x}" for byte in b)
    print(f"    {byte_string}, # {instruction_text}")    
print("])")

PROGRAM_D = bytes([
    0x93, 0x04, 0x30, 0x00, # addi x9 x0 3
    0x13, 0x09, 0xa0, 0x00, # addi x18 x0 10
    0x93, 0x09, 0x04, 0x08, # addi x19 x8 128
    0x63, 0x8c, 0x04, 0x00, # beq x9 x0 24
    0x23, 0xa0, 0x29, 0x01, # sw x18 0(x19)
    0x13, 0x09, 0xa9, 0x00, # addi x18 x18 10
    0x93, 0x89, 0x49, 0x00, # addi x19 x19 4
    0x93, 0x84, 0xf4, 0xff, # addi x9 x9 -1
    0x6f, 0xf0, 0xdf, 0xfe, # jal x0 -20
    0x13, 0x05, 0x04, 0x08, # addi x10 x8 128
    0x93, 0x05, 0x30, 0x00, # addi x11 x0 3
    0xef, 0x00, 0x40, 0x05, # jal x1 84
    0x93, 0x02, 0x04, 0x01, # addi x5 x8 16
    0x23, 0xa0, 0xc2, 0x00, # sw x12 0(x5)
    0x13, 0x03, 0x40, 0x1f, # addi x6 x0 500
    0xb3, 0x23, 0xc3, 0x00, # slt x7 x6 x12
    0x93, 0x02, 0x44, 0x01, # addi x5 x8 20
    0x23, 0xa0, 0x72, 0x00, # sw x7 0(x5)
    0x13, 0x0e, 0x00, 0x0f, # addi x28 x0 240
    0x93, 0x0e, 0xf0, 0x00, # addi x29 x0 15
    0xb3, 0x42, 0xde, 0x01, # xor x5 x28 x29
    0x33, 0x73, 0xde, 0x01, # and x6 x28 x29
    0xb3, 

# Fin del código

In [74]:
# No usada

R_instr = [
	"add","sub", "sll", 
	"sltu", "xor", "srl", "slt",
	"sra", "or", "and",
	"addw", "subw", "sllw",
	"slrw", "sraw", "mul",
	"mulh", "mulu", "mulsu",
	"div", "divu", "rem",
	"remu"
]
I_instr = [
	"addi", "lb", "lw",
	"ld", "lbu", "lhu","lh",
	"lwu", "fence", "fence.i", 
	"slli", "slti", "sltiu", 
	"xori", "slri", "srai",
	"ori", "andi", "addiw",
	"slliw", "srliw", "sraiw", 
	"jalr", "ecall", "ebreak", 
	"csrrw", "csrrs","csrrc", 
	"csrrwi", "csrrsi", "csrrci" 
]
S_instr = [
	"sw", "sb", "sh", 
	"sd"
]
SB_instr = [
	"beq", "bne", "blt", 
	"bge", "bltu", "bgeu"
]

U_instr = ["auipc", "lui"]

UJ_instr = ["jal"]

def get_type(instr: str) -> str:
    if instr in R_instr:  return "R"
    if instr in I_instr:  return "I"
    if instr in S_instr:  return "S"
    if instr in SB_instr: return "SB"
    if instr in U_instr:  return "U"
    if instr in UJ_instr: return "UJ"
    return "UNKNOWN"

if False:
	with open("instr_data.dat", "r") as infile, open("instruction_data.dat", "w") as outfile:
		for line in infile:
			parts = line.strip().split()
			if not parts:
				continue
			instr = parts[0]
			itype = get_type(instr.lower())  # lower() para asegurar coincidencia
			new_line = " ".join([instr.lower(), itype] + parts[1:])
			outfile.write(new_line + "\n")